# Import Statements

In [1]:
import pandas as pd
import numpy as np
import string
import os
import re
import pickle

In [2]:
from sortedcontainers import SortedDict, SortedList, SortedSet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

# Read From Files

In [3]:
def getListOfFiles(directory):
    '''
    Parameters:
        directory: type(string)
        
    returns: list of all files in directory with the full path of file
    '''
    
    list_of_files = []
    
    for file_path in os.listdir(directory):
        full_path = os.path.join(directory, file_path)
        if os.path.isfile(full_path):
            list_of_files.append(full_path)
    
    return list_of_files

# Preprocessing Functions

In [4]:
def lowercase(data):
    '''
    Parameters:
        data: type(string)
    
    returns: lowercase of data
    '''
    
    return data.lower()

In [5]:
def perform_word_tokenize(corpus):
    '''
    Parameters:
        corpus: type(string)
    
    returns word-level tokenization of corpus
    '''
    
    return word_tokenize(corpus)

In [6]:
def remove_stopwords_from_tokens(tokens, stopwords_set):
    '''
    Parameters:
        tokens: type(list)
        stopwords_set: type(set)
    
    returns: tokens without stopwords
    '''
    tokens_sans_stopwords = [x for x in tokens if x not in stopwords_set]
    
    return tokens_sans_stopwords

In [7]:
def remove_punctuation_from_tokens(tokens):
    '''
    Parameters:
        tokens: type(list)
    
    returns: tokens without punctuation
    '''
    tokens_sans_punctuation = [x.translate(str.maketrans('', '', string.punctuation)) for x in tokens]
    
    return tokens_sans_punctuation

In [8]:
def remove_blank_space_tokens(tokens):
    '''
    Parameters:
        tokens: type(list)
    
    returns: tokens without blank tokens
    '''
    tokens_sans_blank_space = [x for x in tokens if x!='']
    
    return tokens_sans_blank_space

In [9]:
def lemmatize_tokens(tokens):
    '''
    Parameters:
        tokens: type(list)
    
    returns: returns unique tokens after lemmatization
    '''
    lemmatizer = WordNetLemmatizer()
    lemmatize_tokens = [lemmatizer.lemmatize(x) for x in tokens]
    unique_lemmatize_tokens = list(dict.fromkeys(lemmatize_tokens))
    
    return unique_lemmatize_tokens

In [10]:
def preprocess(corpus, stopwords_set, preprocess_type):
    # Convert the text to lower case
    lowercase_corpus = lowercase(corpus)
    #print(len(lowercase_corpus))
    
    # Perform word tokenization (word_tokenize also takes care of whitespace)
    word_tokens = perform_word_tokenize(lowercase_corpus)
    #print(len(word_tokens))
    
    # Remove stopwords from tokens
    word_tokens_sans_stopwords = remove_stopwords_from_tokens(word_tokens, stopwords_set)
    #print(len(word_tokens_sans_stopwords))
    
    # Remove punctuation marks from tokens
    word_tokens_sans_punctuation = remove_punctuation_from_tokens(word_tokens_sans_stopwords)
    #print(len(word_tokens_sans_punctuation))
    
    # Remove blank space tokens
    word_tokens_sans_blank_tokens = remove_blank_space_tokens(word_tokens_sans_punctuation)
    #print(len(word_tokens_sans_blank_tokens))
    
    # Lemmatize tokens
    #word_tokens_final = lemmatize_tokens(word_tokens_sans_blank_tokens)
    #print(len(word_tokens_final))
    
    if preprocess_type=='query':
        return word_tokens_sans_blank_tokens
    return sorted(list(dict.fromkeys(word_tokens_sans_blank_tokens)))

# Helper Functions

In [11]:
def create_file_dictionary(list_of_files):
    '''
    Paramteres:
        list_of_files: type(string)
    
    returns: file_dictionary with integer key and path_of_file as value
    '''
    file_dictionary = {}
    for i in range(len(list_of_files)):
        file_dictionary[i] = list_of_files[i]
    
    return file_dictionary

In [12]:
def create_unigram_inverted_index(file_dictionary, stopwords_set):
    # initialize unigram inverted index
    unigram_inverted_index = SortedDict()
    
    # unigram inverted index
    for doc_ID in range(len(file_dictionary)):
        file = open(file_dictionary[doc_ID], 'r', encoding='utf-8', errors='ignore')
        file_corpus = file.read()
        file.close()
        doc_tokens = preprocess(file_corpus, stopwords_set, 'doc')
        for token in doc_tokens:
            if token in unigram_inverted_index:
                unigram_inverted_index[token][0] += 1
                unigram_inverted_index[token][1].add(doc_ID)
            else:
                unigram_inverted_index[token] = [1, SortedSet([doc_ID])]
    
    # Storing unigram inverted index
    uii_file = open('unigram_inverted_index_pickle_file', 'wb')
    pickle.dump(unigram_inverted_index, uii_file)
    uii_file.close()

# Boolean Functions

In [13]:
def xANDy(pos_list_1, pos_list_2):
    result = SortedSet()
    i,j = 0,0
    num_of_operations = 0
    while(i<len(pos_list_1) and j<len(pos_list_2)):
        if pos_list_1[i]==pos_list_2[j]:
            result.add(pos_list_1[i])
            i+=1
            j+=1
        elif pos_list_1[i]<pos_list_2[j]:
            i+=1
        else:
            j+=1
        num_of_operations+=1
    
    return result, num_of_operations

In [14]:
def xORy(pos_list_1, pos_list_2):
    result = SortedSet()
    i,j = 0,0
    num_of_operations = 0
    while(i<len(pos_list_1) and j<len(pos_list_2)):
        if pos_list_1[i]==pos_list_2[j]:
            result.add(pos_list_1[i])
            i+=1
            j+=1
        elif pos_list_1[i]<pos_list_2[j]:
            result.add(pos_list_1[i])
            i+=1
        else:
            result.add(pos_list_2[j])
            j+=1
        num_of_operations+=1
    
    while(i<len(pos_list_1)):
        result.add(pos_list_1[i])
        i+=1
    while(j<len(pos_list_2)):
        result.add(pos_list_2[j])
        j+=1
    
    return result, num_of_operations

In [15]:
def xANDNOTy(pos_list_1, pos_list_2):
    result = SortedSet()
    i,j = 0,0
    num_of_operations = 0
    while(i<len(pos_list_1) and j<len(pos_list_2)):
        if pos_list_1[i]==pos_list_2[j]:
            i+=1
            j+=1
        elif pos_list_1[i]<pos_list_2[j]:
            result.add(pos_list_1[i])
            i+=1
        else:
            j+=1
        num_of_operations+=1
    
    while(i<len(pos_list_1)):
        result.add(pos_list_1[i])
        i+=1
    
    return result, num_of_operations

In [16]:
def xORNOTy(pos_list_1, pos_list_2, u_list):
    num_of_not_operations = 0
    i,j = 0,0
    not_pos_list_2 = SortedSet()
    while(i<len(pos_list_2) and j<len(u_list)):
        if pos_list_2[i]<u_list[j]:
            i+=1
        elif pos_list_2[i]>u_list[j]:
            not_pos_list_2.add(u_list[j])
            j+=1
        else:
            i+=1
            j+=1
        num_of_not_operations+=1
    while(j<len(u_list)):
        not_pos_list_2.add(u_list[j])
        j+=1
    
    result, ops = xORy(pos_list_1, not_pos_list_2)
    return result, num_of_not_operations+ops

# Main

In [35]:
def main():
    # create set of stop words for preprocessing
    stopwords_set = set(stopwords.words('english'))
    
    # Get List of Files in Dataset
    list_of_files = getListOfFiles('Dataset/Humor,Hist,Media,Food/')
    
    # create dictionary of file with docID (integer) as key and full_path of file as value
    file_dictionary = create_file_dictionary(list_of_files)
    
    # create unigram inverted index once and then load pickle file afterwards
    #create_unigram_inverted_index(file_dictionary, stopwords_set)
    
    #Loading pre-processed files
    uii_file = open('unigram_inverted_index_pickle_file', 'rb')
    unigram_inverted_index = pickle.load(uii_file)
    uii_file.close()
    u_list = SortedSet()
    for a in range(len(file_dictionary)):
        u_list.add(a)
    
    N = int(input())
    for q in range(N):
        input_sentence = input()
        input_operation_sequence = input()
        sanitized_query = preprocess(input_sentence, stopwords_set, 'query')
        sanitized_operation_sequence = []
        for op_seq in input_operation_sequence.split(','):
            sanitized_operation_sequence.append(op_seq.lower().strip())
        print(sanitized_query)
        print(sanitized_operation_sequence)
        if(len(sanitized_query)!=len(sanitized_operation_sequence)+1):
            print('invalid query')
            continue
        ptr1,ptr2 = 0,0
        result = unigram_inverted_index[sanitized_query[ptr1]][1]
        num_of_operations = 0
        while(ptr2<len(sanitized_operation_sequence)):
            if sanitized_operation_sequence[ptr2]=='and':
                res, ops = xANDy(result, unigram_inverted_index[sanitized_query[ptr1+1]][1])
            elif sanitized_operation_sequence[ptr2]=='or':
                res, ops = xORy(result, unigram_inverted_index[sanitized_query[ptr1+1]][1])
            elif sanitized_operation_sequence[ptr2]=='and not':
                res, ops = xANDNOTy(result, unigram_inverted_index[sanitized_query[ptr1+1]][1])
            elif sanitized_operation_sequence[ptr2]=='or not':
                res, ops = xORNOTy(result, unigram_inverted_index[sanitized_query[ptr1+1]][1], u_list)
            ptr1+=1
            ptr2+=1
            result = res
            num_of_operations += ops
        print('Number of documents matched: {}'.format(len(result)))
        print('Number of comparisons required: {}'.format(num_of_operations))
        print('Documents: \n')
        for res_doc in result:
            print(file_dictionary[res_doc])

In [38]:
if __name__ == "__main__":
    main()

1
brutus$ -- chicken kitchen
or not, and not
['brutus', 'chicken', 'kitchen']
['or not', 'and not']
Number of documents matched: 988
Number of comparisons required: 3053
Documents: 

Dataset/Humor,Hist,Media,Food/1st_aid.txt
Dataset/Humor,Hist,Media,Food/a-team
Dataset/Humor,Hist,Media,Food/abbott.txt
Dataset/Humor,Hist,Media,Food/aboutada.txt
Dataset/Humor,Hist,Media,Food/acetab1.txt
Dataset/Humor,Hist,Media,Food/aclamt.txt
Dataset/Humor,Hist,Media,Food/acne1.txt
Dataset/Humor,Hist,Media,Food/acronym.lis
Dataset/Humor,Hist,Media,Food/acronym.txt
Dataset/Humor,Hist,Media,Food/adameve.hum
Dataset/Humor,Hist,Media,Food/adcopy.hum
Dataset/Humor,Hist,Media,Food/addrmeri.txt
Dataset/Humor,Hist,Media,Food/admin.txt
Dataset/Humor,Hist,Media,Food/adrian_e.faq
Dataset/Humor,Hist,Media,Food/adt_miam.txt
Dataset/Humor,Hist,Media,Food/aeonint.txt
Dataset/Humor,Hist,Media,Food/age.txt
Dataset/Humor,Hist,Media,Food/aggie.txt
Dataset/Humor,Hist,Media,Food/aids.txt
Dataset/Humor,Hist,Media,Food/airlin